# Track the fish over consequtive images
This notebook shows how to track the fish over consequtive images using the tracking algorithm.

## Install software
To install the software on your own computer, follow the steps provided in the [readme](https://github.com/Rick-v-E/automatic_discard_registration/blob/master/README.md). If running on Google Colab, clone the GIT repository and install it's dependencies:

In [ ]:
%%shell

# Check if the repository is already available, if not, clone and install
if [ ! -d .git ]
then
  git clone --recurse-submodules https://github.com/Rick-v-E/automatic_discard_registration.git
  pip install -r automatic_discard_registration/requirements.txt
  pip install -r automatic_discard_registration/detection/yolov3/requirements.txt
  pip install automatic_discard_registration/detection/apex
  pip install gdown
else
  git pull
fi

If you installed the software in the previous step, enter the repository:

In [ ]:
%cd automatic_discard_registration

## Setup dataset
The complete dataset can be downloaded from [4TU.ResearchData](https://doi.org/10.4121/16622566.v1). To use this dataset, extract both `fdf_images.zip` and `results.zip` in the [data](https://github.com/Rick-v-E/automatic_discard_registration/tree/master/data) folder.

For use on Google Colab, we have created a smaller subset of the data. This dataset contains only part of the images of the complete dataset, but contains all result from the complete dataset.

---
**IMPORTANT**

Execute only one of the three cells below! Each cell contains a method to import the data, if one method fails, use another method. If the method succeed, go to the next section in this notebook.

---

**METHOD 1** Download and extract the sample dataset (this will take around 5-10 minutes):

In [ ]:
!gdown --id 1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ
!unzip -q fdf_sample_dataset.zip -d data
!rm fdf_sample_dataset.zip

**METHOD 2** Download the [sample dataset](https://drive.google.com/file/d/1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ/view?usp=sharing) manually and upload it to Google Colab in the `automatic_discard_registration` opening the files tab and right click on the folder name:

![Manual upload image](colab_manual_upload.png)

After uploading, extract the dataset:

In [ ]:
!unzip -q fdf_sample_dataset.zip -d data
!rm fdf_sample_dataset.zip

**METHOD 3** Download the [sample dataset](https://drive.google.com/file/d/1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ/view?usp=sharing) and upload it to your personal Google Drive account. Connect this account to Google Colab:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q ../drive/MyDrive/fdf_sample_dataset.zip -d data

Check if the dataset is loaded correctly:

In [ ]:
from pathlib import Path

DATA_PATH = Path("data")
NEEDED_FOLDERS = ["fdf_images", "results"]

# Check if all folders are correct
if not all([(DATA_PATH / f).is_dir() for f in NEEDED_FOLDERS]):
    print("Could not find all data folders! Did you extract both fdf_images.zip and results.zip in the data folder?")

To get the same results as in the paper, use the complete dataset. Upload the dataset to your Google Drive and [mount](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166) this folder to your Google Colab environment. 

## Setup tracking notebook
Start by importing the dependencies:

In [ ]:
%matplotlib notebook

import cv2

from pathlib import Path
from IPython.display import HTML
from tqdm.notebook import tqdm

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from common.nb_utils import show_image_with_trackers
from common.io import load_detection_file, load_image_file_names, write_trackers_to_json
from tracking.multi_tracker import MultiTracker
from tracking.metrics.iou import IoUMetric

Setup all paths to the detection files and images folders. On the example dataset, not all detection files and image folders are available. If you are running on the complete dataset, you can uncomment these lines.

In [ ]:
path_dict = {
    "20191018_run_1_images": Path("data/fdf_images/images/test_sequences/20191018_run_1"),
    # "20191018_run_2_images": Path("data/fdf_images/images/test_sequences/20191018_run_2"),
    # "20191018_run_3_images": Path("data/fdf_images/images/test_sequences/20191018_run_3"),
    # "20191018_run_4_images": Path("data/fdf_images/images/test_sequences/20191018_run_4"),
    # "20191025_run_1_images": Path("data/fdf_images/images/test_sequences/20191025_run_1"),
    # "20191025_run_2_images": Path("data/fdf_images/images/test_sequences/20191025_run_2"),
    # "20191025_run_3_images": Path("data/fdf_images/images/test_sequences/20191025_run_3"),
    # "20191025_run_4_images": Path("data/fdf_images/images/test_sequences/20191025_run_4"),
    # "20191101_run_1_images": Path("data/fdf_images/images/test_sequences/20191101_run_1"),
    # "20191101_run_2_images": Path("data/fdf_images/images/test_sequences/20191101_run_2"),
    # "20191101_run_3_images": Path("data/fdf_images/images/test_sequences/20191101_run_3"),
    # "20191101_run_4_images": Path("data/fdf_images/images/test_sequences/20191101_run_4"),
    "20191018_run_1_detections": Path("data/results/EM_comparison/20191018_run_1_detections.json"),
    # "20191018_run_2_detections": Path("data/results/EM_comparison/20191018_run_2_detections.json"),
    # "20191018_run_3_detections": Path("data/results/EM_comparison/20191018_run_3_detections.json"),
    # "20191018_run_4_detections": Path("data/results/EM_comparison/20191018_run_4_detections.json"),
    # "20191025_run_1_detections": Path("data/results/EM_comparison/20191025_run_1_detections.json"),
    # "20191025_run_2_detections": Path("data/results/EM_comparison/20191025_run_2_detections.json"),
    # "20191025_run_3_detections": Path("data/results/EM_comparison/20191025_run_3_detections.json"),
    # "20191025_run_4_detections": Path("data/results/EM_comparison/20191025_run_4_detections.json"),
    # "20191101_run_1_detections": Path("data/results/EM_comparison/20191101_run_1_detections.json"),
    # "20191101_run_2_detections": Path("data/results/EM_comparison/20191101_run_2_detections.json"),
    # "20191101_run_3_detections": Path("data/results/EM_comparison/20191101_run_3_detections.json"),
    # "20191101_run_4_detections": Path("data/results/EM_comparison/20191101_run_4_detections.json"),
}

assert all([f.is_file() for n, f in path_dict.items() if "detections" in n])
assert all([f.is_dir() for n, f in path_dict.items() if "images" in n])

Load the first sequence:

In [ ]:
images_dict = load_image_file_names(path_dict["20191018_run_1_images"])
detection_dict = load_detection_file(path_dict["20191018_run_1_detections"])

print(f"Loaded { len(images_dict) } images with { len(detection_dict) } annotations")

Define the evaluation metric for the tracker:

In [ ]:
metric = IoUMetric
metric.threshold = 0.2

Define the tracker for the first sequence:

In [ ]:
first_image = list(images_dict.values())[0]  # Needed to initialize the ORB translation estimator
tracker = MultiTracker(first_image, metric=metric, min_create_objectness_threshold=0.01, max_age=1, n_init=2)

Loop over all detections and track the fishes over the images. In real-time, this will work faster since we don't have to load the image (which is time-consuming).

In [ ]:
min_objectness_threshold = 0.01

for image_name, image_path in tqdm(images_dict.items(), desc="Tracking fishes"):
    image = cv2.imread(str(image_path))

    if image is None:
        print(f"Could not read image { image_path }!")
        continue

    detections = [dt for dt in detection_dict[image_name] if dt.is_certain(min_objectness_threshold)]
    
    # Predict the position of the detections in a next image
    tracker.predict(image_name)

    # Update the position
    tracker.update(detections, image, image_name)

trackers = tracker.get_trackers()

print(f"Tracked { len(trackers) } fishes over { len(images_dict) } images...")

Save all the trackers as a `.json` file:

In [ ]:
trackers_file = Path("data/20191018_run_1_tracking.json")
write_trackers_to_json(trackers_file, trackers)

Make a visualisation of the images:

In [ ]:
matplotlib.rcParams["animation.embed_limit"] = 2e5
fig, ax = plt.subplots()
ims = []

trks = load_detection_file(trackers_file)
for i, (name, path) in tqdm(enumerate(images_dict.items()), desc="Rendering annimation"):
    image = cv2.imread(str(path))
    im = show_image_with_trackers(image, trks[name], ax)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=1000)
HTML(ani.to_html5_video())

Track all the sequences if you are using the complete dataset:

In [ ]:
sequences = ["20191018_run_2", "20191018_run_3", "20191018_run_4", "20191025_run_1", "20191025_run_2", "20191025_run_3", "20191025_run_4", "20191101_run_1", "20191101_run_2", "20191101_run_3", "20191101_run_4"]
for subset_name in tqdm(sequences, desc="Tracking subset"):
    images_dict = load_image_file_names(path_dict[f"{ subset_name }_images"])
    detection_dict = load_detection_file(path_dict[f"{ subset_name }_detections"])

    metric = IoUMetric
    metric.threshold = 0.2

    first_image = list(images_dict.values())[0]  # Needed to initialize the ORB translation estimator
    tracker = MultiTracker(first_image, metric=metric, min_create_objectness_threshold=0.01, max_age=1, n_init=2)

    min_objectness_threshold = 0.01

    for image_name, image_path in tqdm(images_dict.items(), desc="Tracking fishes"):
        image = cv2.imread(str(image_path))

        if image is None:
            print(f"Could not read image { image_path }!")
            continue

        detections = [dt for dt in detection_dict[image_name] if dt.is_certain(min_objectness_threshold)]
        
        # Predict the position of the detections in a next image
        tracker.predict(image_name)

        # Update the position
        tracker.update(detections, image, image_name)

    trackers = tracker.get_trackers()

    trackers_file = Path(f"data/{ subset_name }_tracking.json")
    write_trackers_to_json(trackers_file, trackers) 


All tracking files are now saved in the `data` folder. In the [evaluation notebook](https://github.com/Rick-v-E/automatic_discard_registration/blob/master/evaluate.ipynb), we use 'our' tracking files.